# 指標互補

In [9]:
import backtrader as bt


class OvernightReversalStrategy(bt.Strategy):
    """
    隔日雙邊策略（修正版）：
    - 收盤判斷訊號 → 不下單，只設定 flag
    - 下一根開盤（next_open）才：
      → 全倉進場 or 出場
    """

    params = dict(
        sma_period=20,
        adx_period=14,
        adx_min=20,
        printlog=False,
        risk_fraction=0.99,     # 全倉比例，預留手續費空間
    )

    def log(self, txt, dt=None):
        if self.p.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()} - {txt}')

    def __init__(self):
        data = self.datas[0]

        self.order = None
        self.pending_signal = None  # 'long' or 'short'

        # ----- Indicators -----
        self.sma = bt.indicators.SimpleMovingAverage(data.close,
                                                     period=self.p.sma_period)

        self.macd = bt.indicators.MACD(data.close)
        self.macd_signal = self.macd.signal

        self.adx = bt.indicators.ADX(data, period=self.p.adx_period)

    # ===== 收盤判斷訊號，不在這裡成交 =====
    def next(self):
        if self.order:
            return

        data = self.datas[0]
        close = data.close[0]
        position_size = self.position.size

        # 先判斷是否需要反轉出場
        if position_size > 0:  # Long
            if (
                close < self.sma[0] or
                self.macd.macd[0] < self.macd_signal[0] or
                self.adx[0] < self.p.adx_min
            ):
                self.pending_signal = 'exit'
                return

        elif position_size < 0:  # Short
            if (
                close > self.sma[0] or
                self.macd.macd[0] > self.macd_signal[0] or
                self.adx[0] < self.p.adx_min
            ):
                self.pending_signal = 'exit'
                return

        # 如果無持倉→判斷新進場
        if position_size == 0:
            if (
                close > self.sma[0] and
                self.macd.macd[0] > self.macd_signal[0] and
                self.adx[0] > self.p.adx_min
            ):
                self.pending_signal = 'long'
            elif (
                close < self.sma[0] and
                self.macd.macd[0] < self.macd_signal[0] and
                self.adx[0] > self.p.adx_min
            ):
                self.pending_signal = 'short'

    # ===== 在開盤成交（需配合 cheat_on_open=True） =====
    def next_open(self):
        if self.pending_signal is None:
            return

        data = self.datas[0]
        open_price = data.open[0]
        value = self.broker.get_value()
        size = int((value * self.p.risk_fraction) / open_price)

        # 先清掉未完成訂單
        if self.order:
            return

        # EXIT
        if self.pending_signal == 'exit' and self.position:
            self.log(f'EXIT @ {open_price:.2f}')
            self.order = self.close()

        # LONG
        elif self.pending_signal == 'long':
            if self.position.size < 0:
                self.order = self.close()  # 平空再開多
            self.log(f'LONG ENTRY @ {open_price:.2f} ALL IN')
            self.order = self.buy(size=size)

        # SHORT
        elif self.pending_signal == 'short':
            if self.position.size > 0:
                self.order = self.close()  # 平多再開空
            self.log(f'SHORT ENTRY @ {open_price:.2f} ALL IN')
            self.order = self.sell(size=size)

        # 訊號處理完畢
        self.pending_signal = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            action = 'BUY' if order.isbuy() else 'SELL'
            self.log(f'{action} EXECUTED @ {order.executed.price:.2f}')

        self.order = None

    def stop(self):
        if self.p.printlog:
            self.log(f'Final Value: {self.broker.getvalue():.2f}')



In [15]:
if __name__ == '__main__':
    cerebro = bt.Cerebro(cheat_on_open=True)  # 重點：允許在 open 成交前下單
    cerebro.addstrategy(OvernightReversalStrategy,
                        sma_period=20,
                        adx_period=14,
                        adx_min=20,
                        printlog=True)
    
    import yfinance as yf
    
    data = yf.download('2330.TW', start='2020-01-01')
    data.columns = data.columns.droplevel(1)
    cerebro.adddata(bt.feeds.PandasData(dataname=data))

    cerebro.broker.setcash(100000.0)
    cerebro.run()
    
    print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')


/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_54673/3854701217.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('2330.TW', start='2020-01-01')
[*********************100%***********************]  1 of 1 completed

2020-03-12 - SHORT ENTRY @ 265.79 ALL IN
2020-03-12 - SELL EXECUTED @ 265.79
2020-03-27 - EXIT @ 254.91
2020-03-27 - BUY EXECUTED @ 254.91
2020-04-08 - LONG ENTRY @ 255.81 ALL IN
2020-04-08 - BUY EXECUTED @ 255.81
2020-04-16 - EXIT @ 254.01
2020-04-16 - SELL EXECUTED @ 254.01
2020-06-29 - LONG ENTRY @ 284.09 ALL IN
2020-06-29 - BUY EXECUTED @ 284.09
2020-06-30 - EXIT @ 283.64
2020-06-30 - SELL EXECUTED @ 283.64
2020-07-06 - LONG ENTRY @ 300.83 ALL IN
2020-07-06 - BUY EXECUTED @ 300.83
2020-08-12 - EXIT @ 381.81
2020-08-12 - SELL EXECUTED @ 381.81
2020-08-21 - SHORT ENTRY @ 381.35 ALL IN
2020-08-21 - SELL EXECUTED @ 381.35
2020-08-25 - EXIT @ 392.21
2020-08-25 - BUY EXECUTED @ 392.21
2020-09-01 - SHORT ENTRY @ 389.04 ALL IN
2020-09-01 - SELL EXECUTED @ 389.04
2020-09-02 - EXIT @ 399.00
2020-09-02 - BUY EXECUTED @ 399.00
2020-09-07 - SHORT ENTRY @ 387.24 ALL IN
2020-09-07 - SELL EXECUTED @ 387.24
2020-09-09 - EXIT @ 384.52
2020-09-09 - BUY EXECUTED @ 384.52
2020-09-10 - SHORT ENTRY @ 391

In [16]:
import backtrader as bt


class OvernightMultiTFStrategy(bt.Strategy):
    """
    隔日雙邊策略 + 週線趨勢過濾

    - 日線：負責進出場與隔日下單
    - 週線：50 SMA 判斷大方向
      * 週線收盤 > 週線 SMA → 只做多
      * 週線收盤 < 週線 SMA → 只做空

    - 收盤判斷訊號 → pending_signal
    - 下一根 K 線開盤（next_open）全倉進出
    """

    params = dict(
        sma_period=20,          # 日線 SMA
        adx_period=14,
        adx_min=20,
        weekly_sma_period=50,   # 週線 SMA
        risk_fraction=0.99,     # 全倉比例
        printlog=False,
    )

    def log(self, txt, dt=None):
        if self.p.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()} - {txt}')

    def __init__(self):
        # ===== 資料引用 =====
        self.data_daily = self.datas[0]    # 主交易時間框架：日線
        self.data_weekly = self.datas[1]   # 趨勢確認：週線

        # 掛單 & 訊號狀態
        self.order = None
        self.pending_signal = None   # 'long' / 'short' / 'exit'

        # ===== 日線指標：SMA + MACD + ADX =====
        self.sma = bt.indicators.SimpleMovingAverage(
            self.data_daily.close,
            period=self.p.sma_period
        )

        self.macd = bt.indicators.MACD(self.data_daily.close)
        self.macd_signal = self.macd.signal

        self.adx = bt.indicators.ADX(self.data_daily,
                                     period=self.p.adx_period)

        # ===== 週線指標：50 SMA 趨勢 =====
        self.weekly_sma = bt.indicators.SimpleMovingAverage(
            self.data_weekly.close,
            period=self.p.weekly_sma_period
        )

    # ========= 收盤：決定要不要發出「明天要做什麼」的訊號 =========
    def next(self):
        # 有掛單就先不要下新指令
        if self.order:
            return

        # 週線 SMA 尚未有足夠資料 → 不交易
        if len(self.weekly_sma) < self.p.weekly_sma_period:
            return

        daily_close = self.data_daily.close[0]
        position_size = self.position.size

        # ===== 週線趨勢方向 =====
        weekly_close = self.data_weekly.close[0]
        weekly_bull = weekly_close > self.weekly_sma[0]
        weekly_bear = weekly_close < self.weekly_sma[0]
        # 如果剛好貼在 SMA 附近，你也可以選擇當作「不明確，不下單」

        # ===== 先看是否需要出場（市場反轉 or 趨勢轉弱）=====
        if position_size > 0:  # Long 部位
            if (
                daily_close < self.sma[0] or
                self.macd.macd[0] < self.macd_signal[0] or
                self.adx[0] < self.p.adx_min
            ):
                self.pending_signal = 'exit'
                return

        elif position_size < 0:  # Short 部位
            if (
                daily_close > self.sma[0] or
                self.macd.macd[0] > self.macd_signal[0] or
                self.adx[0] < self.p.adx_min
            ):
                self.pending_signal = 'exit'
                return

        # ===== 沒部位時，才考慮新進場 =====
        if position_size == 0:
            # 日線多頭訊號
            daily_long_signal = (
                daily_close > self.sma[0] and
                self.macd.macd[0] > self.macd_signal[0] and
                self.adx[0] > self.p.adx_min
            )

            # 日線空頭訊號
            daily_short_signal = (
                daily_close < self.sma[0] and
                self.macd.macd[0] < self.macd_signal[0] and
                self.adx[0] > self.p.adx_min
            )

            # 結合週線大方向過濾
            if weekly_bull and daily_long_signal:
                self.pending_signal = 'long'
            elif weekly_bear and daily_short_signal:
                self.pending_signal = 'short'
            # 若週線多、日線空 或 週線空、日線多 → 直接略過，不逆勢下單

    # ========= 開盤：依照昨天收盤產生的 pending_signal 來「全倉成交」 =========
    def next_open(self):
        if self.pending_signal is None:
            return
        if self.order:
            return

        open_price = self.data_daily.open[0]
        value = self.broker.get_value()
        size = int((value * self.p.risk_fraction) / open_price)

        # EXIT
        if self.pending_signal == 'exit' and self.position:
            self.log(f'EXIT @ {open_price:.2f}')
            self.order = self.close()

        # LONG
        elif self.pending_signal == 'long':
            if self.position.size < 0:
                self.log(f'CLOSE SHORT @ {open_price:.2f}')
                self.order = self.close()
            self.log(f'LONG ENTRY @ {open_price:.2f}, SIZE={size}')
            self.order = self.buy(size=size)

        # SHORT
        elif self.pending_signal == 'short':
            if self.position.size > 0:
                self.log(f'CLOSE LONG @ {open_price:.2f}')
                self.order = self.close()
            self.log(f'SHORT ENTRY @ {open_price:.2f}, SIZE={size}')
            self.order = self.sell(size=size)

        # 訊號執行完畢
        self.pending_signal = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            action = 'BUY' if order.isbuy() else 'SELL'
            self.log(f'{action} EXECUTED @ {order.executed.price:.2f}')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'TRADE PROFIT, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}')

    def stop(self):
        if self.p.printlog:
            self.log(f'Final Value: {self.broker.getvalue():.2f}')


In [18]:
import backtrader as bt
from datetime import datetime


if __name__ == '__main__':
    cerebro = bt.Cerebro(cheat_on_open=True)

    import yfinance as yf
    
    df = yf.download('2330.TW', start='2020-01-01')
    df.columns = df.columns.droplevel(1)
    data = bt.feeds.PandasData(
        dataname=df,
        timeframe=bt.TimeFrame.Days
        )

    cerebro = bt.Cerebro(cheat_on_open=True)

    # 加入日線資料
    cerebro.adddata(data)  # datas[0]

    # Backtrader resample → 週線資料
    cerebro.resampledata(
        data,
        timeframe=bt.TimeFrame.Weeks,
        compression=1
    )  # datas[1]

    cerebro.addstrategy(OvernightMultiTFStrategy,
                        sma_period=20,
                        adx_period=14,
                        adx_min=20,
                        weekly_sma_period=50,
                        printlog=True)

    cerebro.broker.setcash(100000.0)
    cerebro.run()
    cerebro.plot()


/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_54673/4269222285.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('2330.TW', start='2020-01-01')
[*********************100%***********************]  1 of 1 completed


2020-12-15 - LONG ENTRY @ 461.23, SIZE=214
2020-12-15 - BUY EXECUTED @ 461.23
2020-12-16 - EXIT @ 463.05
2020-12-16 - SELL EXECUTED @ 463.05
2020-12-16 - TRADE PROFIT, Gross: 389.36, Net: 389.36
2021-01-04 - LONG ENTRY @ 484.52, SIZE=205
2021-01-04 - BUY EXECUTED @ 484.52
2021-01-28 - EXIT @ 548.51
2021-01-28 - SELL EXECUTED @ 548.51
2021-01-28 - TRADE PROFIT, Gross: 13118.55, Net: 13118.55
2021-05-26 - LONG ENTRY @ 538.82, SIZE=208
2021-05-26 - BUY EXECUTED @ 538.82
2021-06-15 - EXIT @ 557.18
2021-06-15 - SELL EXECUTED @ 557.18
2021-06-15 - TRADE PROFIT, Gross: 3818.58, Net: 3818.58
2021-09-07 - LONG ENTRY @ 584.38, SIZE=198
2021-09-07 - BUY EXECUTED @ 584.38
2021-09-16 - EXIT @ 558.34
2021-09-16 - SELL EXECUTED @ 558.34
2021-09-16 - TRADE PROFIT, Gross: -5156.78, Net: -5156.78
2022-01-03 - LONG ENTRY @ 575.79, SIZE=192
2022-01-03 - BUY EXECUTED @ 575.79
2022-01-24 - EXIT @ 594.39
2022-01-24 - SELL EXECUTED @ 594.39
2022-01-24 - TRADE PROFIT, Gross: 3571.94, Net: 3571.94
2022-03-08 - 

<IPython.core.display.Javascript object>